In [22]:
import time
from bs4 import BeautifulSoup
import pandas as pd
import requests
import json

ITEMS_COUNT = 100

In [23]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36',
    
}

timeout_settings = (60, 60)

In [24]:
middle_category_nums = ['367', '596', '2074', '595', '980', '981', 
                        '1477', '657', '1004', '2080', '1003', '1002',
                        '2224']

In [25]:
def get_response(url, headers):

    # GET 요청
    response = requests.get(url, headers=headers, timeout=timeout_settings)
    soup = BeautifulSoup(response.text, 'lxml')
    time.sleep(0.5)

    return soup


In [26]:
def get_item_info(goods):
    # 상품 id
    goodsNo = goods['goodsNo']

    # 리뷰 갯수
    goodsRevCnt = goods['goodsRevCnt']

    return [goodsNo, goodsRevCnt]

In [27]:
def get_items(num):
    url = f'https://www.thehandsome.com/api/display/1/ko/category/categoryGoodsList?dispMediaCd=10&sortGbn=20&pageSize={ITEMS_COUNT}&pageNo=1&norOutletGbCd=J&dispCtgNo={num}&productListLayout=4&theditedYn=N'

    goods_info = []

    soup = get_response(url, headers)
    info = soup.string
    goods_in_page = json.loads(info)['payload']['goodsList']

    for goods in goods_in_page:
        
        goods_data = get_item_info(goods)
        goods_info.append(goods_data)
        print(len(goods_info), goods_data)

    return goods_info

In [28]:
def get_one_review(review):
    
    # 상품 id
    goodsNo = review['goodsNo']

    # 리뷰 id
    review_id = review['revNo']

    # 내용
    revCont = review['revCont']

    # 키
    try:
        height = review['revPrfleList'][0]['mbrPrfleValNm']
    except:
        height = None
    
    # 평소 사이즈
    try:
        nor_size = review['revPrfleList'][1]['mbrPrfleValNm']
    except:
        nor_size = None

    # 체형 정보
    form_info = {'height': height,
                 'nor_size': nor_size}
    

    data = {
        'goodsNo': goodsNo,
        'review_id': review_id,
        'revCont': revCont,
        'form_info': form_info
    }

    return data

In [32]:
def get_review_data(goodsNo, goodsRevCnt):
    url = f'https://www.thehandsome.com/api/goods/1/ko/goods/{goodsNo}/reviews?sortTypeCd=latest&revGbCd=&pageSize={goodsRevCnt}&pageNo=1'

    one_goods_reviews = []

    soup = get_response(url, headers)
    reviews = soup.string

    try:
        reviews = json.loads(reviews)['payload']['revAllList']

        for review in reviews:
            one_review = get_one_review(review)
            one_goods_reviews.append(one_review)
    except:
        pass
    
    print(f'{goodsNo} 상품 리뷰 수: {len(one_goods_reviews)}', one_goods_reviews)


    return one_goods_reviews

    

In [33]:
def main():
    all_info = []
    for num in middle_category_nums:
        middle_category_info = get_items(num)
        all_info += middle_category_info

    all_reviews = []

    for goodsNo, goodsRevCnt in all_info:
        one_goods_reviews = get_review_data(goodsNo, goodsRevCnt)
        all_reviews += one_goods_reviews


In [34]:
if __name__ == '__main__':
    main()  

1 ['TM2E5KCDC81W', 30]
2 ['SY2E3WJMTE1WM2', 61]
3 ['CM2E3WJC158WP1', 6]
4 ['TM2E4KCD261W', 22]
5 ['MN2E5WOT584W', 6]
6 ['LC2E3WOT445WM2', 4]
7 ['SY2E1WOT718W', 5]
8 ['SY2E3WJMTE1W', 44]
9 ['CM2E3KCD526W', 81]
10 ['TM2E1WJC823WP6', 14]
11 ['CM2E5KOT545W', 7]
12 ['LC2E3KCD654W', 22]
13 ['CM2E4KOT531WM1', 25]
14 ['CM2E5KCD939W', 11]
15 ['MN2E4WJC467W', 5]
16 ['SY2E0WJM652WM2', 6]
17 ['IL2E4KVT341N', 20]
18 ['TM2E3WJC842W', 39]
19 ['CM2E4KCD013W', 26]
20 ['LC2E4WJC965W', 1]
21 ['SY2E4WJC515W', 1]
22 ['CM2E4WOT128W', 1]
23 ['O22E4WJM596W', 5]
24 ['TM2E3NOT950WP1', 0]
25 ['IL2E4KOT244W', 0]
26 ['SJ2E4WJC232W', 0]
27 ['SY2E3WOT614W', 3]
28 ['TM2E3KCD241W', 26]
29 ['TM2E4KCDB61W', 3]
30 ['CM2E1KOT510W', 10]
31 ['CM2E5KOT539W', 1]
32 ['SJ2E3WJC226WM', 0]
33 ['SY2E1NOT718W', 80]
34 ['SY2E3WJMTE1WM4', 1]
35 ['CM2E3KCD522W', 32]
36 ['CM2E4KOT530WM1', 4]
37 ['CM2E4KOT536W', 1]
38 ['CM2E5KOT543W', 29]
39 ['LC2E4KCD666W', 1]
40 ['MN2E3WJC449W', 3]
41 ['OB2E4KCD741W', 7]
42 ['TM2E1WOT521WP4', 6]
43 ['

KeyboardInterrupt: 